In [1]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
for i in range(len(y_train_valid)):
    if(y_train_valid[i] == 769):
        y_train_valid[i] = 0
    if(y_train_valid[i] == 770):
        y_train_valid[i] = 1
    if(y_train_valid[i] == 771):
        y_train_valid[i] = 2
    if(y_train_valid[i] == 772):
        y_train_valid[i] = 3

In [4]:
for i in range(len(y_test)):
    if(y_test[i] == 769):
        y_test[i] = 0
    if(y_test[i] == 770):
        y_test[i] = 1
    if(y_test[i] == 771):
        y_test[i] = 2
    if(y_test[i] == 772):
        y_test[i] = 3

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datatype = torch.float
torch.cuda.device(device)
print(device)

cuda:0


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Temporal Convolution
        self.conv1 = nn.Conv2d(1, 50, (1, 50), stride = (1, 3))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(50)
        # ELU
        self.elu1 = nn.ELU()
        # Spatial Convolution
        self.conv2 = nn.Conv2d(50, 50, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(50)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(50 * 17, 4)
        # Softmax
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [8]:
cnn = CNN().to(device)

In [9]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [11]:
batch_size = 500
num_iters = 1
num_epoch = 1000
epoch_loss = [0.0 for i in range(num_epoch)]
train_accuracy = [0.0 for i in range(num_epoch)]
validation_accuracy = [0.0 for i in range(num_epoch)]
for epoch in range(num_epoch):
    counter = 0
    for train_index, valid_index in kf.split(X_train_valid):
        X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
        y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
        for it in np.arange(num_iters):
            index = np.random.choice(X_train_cur.shape[0], batch_size)
            X_batch = X_train_cur[index, :, :, :]
            y_batch = y_train_cur[index]
            optimizer.zero_grad()
            outputs = cnn(X_batch)

            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss[epoch] += loss

            counter += 1
            
        # Validation Accuracy for final K-fold
        output_val = cnn(X_valid_cur)
        prediction_val = torch.argmax(output_val, axis = 1)
        accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
        print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
        validation_accuracy[epoch] += accuracy_val
        del output_val
        del prediction_val
        del X_valid_cur
        del y_valid_cur
        del accuracy_val
        torch.cuda.empty_cache()

        # Training Accuracy for final K-fold
        output_train = cnn(X_train_cur)
        prediction_train = torch.argmax(output_train, axis = 1)
        accuracy_train = float(prediction_train.eq(y_train_cur).sum()) / float(y_train_cur.shape[0])
        train_accuracy[epoch] += accuracy_train

        del output_train
        del prediction_train
        del X_train_cur
        del y_train_cur
        del accuracy_train
        torch.cuda.empty_cache()
    
    

    epoch_loss[epoch] /= counter 
    print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))
train_accuracy = [i / 4.0 for i in train_accuracy]
validation_accuracy = [i / 4.0 for i in validation_accuracy]
# batch_size = 500
# num_iters = 1
# num_epoch = 1000
# epoch_loss = [0.0 for i in range(num_epoch)]

# for epoch in range(num_epoch):
#     counter = 0
#     for train_index, valid_index in kf.split(X_train_valid):
#         X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
#         X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
#         y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
#         y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
#         for it in np.arange(num_iters):
#             index = np.random.choice(X_train_cur.shape[0], batch_size)
#             X_batch = X_train_cur[index, :, :, :]
#             y_batch = y_train_cur[index]
#             optimizer.zero_grad()
#             outputs = cnn(X_batch)

#             loss = criterion(outputs, y_batch)
#             loss.backward()
#             optimizer.step()
#             epoch_loss[epoch] += loss

#             counter += 1
#         # Validation Accuracy for current K-fold
#         output_val = cnn(X_valid_cur)
#         prediction_val = torch.argmax(output_val, axis = 1)
#         accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
#         print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
#     epoch_loss[epoch] /= counter 
#     print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))

Current Epoch: 0, Validation Accuracy: 0.3629489603024575
Current Epoch: 0, Validation Accuracy: 0.28733459357277885
Current Epoch: 0, Validation Accuracy: 0.39697542533081287
Current Epoch: 0, Validation Accuracy: 0.36174242424242425
Current Epoch: 0, Epoch Loss: 1.3676408529281616
Current Epoch: 1, Validation Accuracy: 0.40831758034026466
Current Epoch: 1, Validation Accuracy: 0.3988657844990548
Current Epoch: 1, Validation Accuracy: 0.4877126654064272
Current Epoch: 1, Validation Accuracy: 0.4564393939393939
Current Epoch: 1, Epoch Loss: 1.3166965246200562
Current Epoch: 2, Validation Accuracy: 0.44045368620037806
Current Epoch: 2, Validation Accuracy: 0.4612476370510397
Current Epoch: 2, Validation Accuracy: 0.5198487712665406
Current Epoch: 2, Validation Accuracy: 0.4981060606060606
Current Epoch: 2, Epoch Loss: 1.2619822025299072
Current Epoch: 3, Validation Accuracy: 0.45179584120982985
Current Epoch: 3, Validation Accuracy: 0.5066162570888468
Current Epoch: 3, Validation Accura

Current Epoch: 29, Validation Accuracy: 0.7126654064272212
Current Epoch: 29, Validation Accuracy: 0.6710775047258979
Current Epoch: 29, Validation Accuracy: 0.722117202268431
Current Epoch: 29, Validation Accuracy: 0.7329545454545454
Current Epoch: 29, Epoch Loss: 1.0344792604446411
Current Epoch: 30, Validation Accuracy: 0.7107750472589792
Current Epoch: 30, Validation Accuracy: 0.6691871455576559
Current Epoch: 30, Validation Accuracy: 0.7391304347826086
Current Epoch: 30, Validation Accuracy: 0.7121212121212122
Current Epoch: 30, Epoch Loss: 1.0317063331604004
Current Epoch: 31, Validation Accuracy: 0.7164461247637051
Current Epoch: 31, Validation Accuracy: 0.6862003780718336
Current Epoch: 31, Validation Accuracy: 0.7391304347826086
Current Epoch: 31, Validation Accuracy: 0.7443181818181818
Current Epoch: 31, Epoch Loss: 1.0372875928878784
Current Epoch: 32, Validation Accuracy: 0.7277882797731569
Current Epoch: 32, Validation Accuracy: 0.6616257088846881
Current Epoch: 32, Valida

Current Epoch: 58, Validation Accuracy: 0.77882797731569
Current Epoch: 58, Validation Accuracy: 0.7580340264650284
Current Epoch: 58, Validation Accuracy: 0.8241965973534972
Current Epoch: 58, Validation Accuracy: 0.7803030303030303
Current Epoch: 58, Epoch Loss: 0.9574000835418701
Current Epoch: 59, Validation Accuracy: 0.7996219281663516
Current Epoch: 59, Validation Accuracy: 0.7334593572778828
Current Epoch: 59, Validation Accuracy: 0.8090737240075614
Current Epoch: 59, Validation Accuracy: 0.8011363636363636
Current Epoch: 59, Epoch Loss: 0.9602708220481873
Current Epoch: 60, Validation Accuracy: 0.8034026465028355
Current Epoch: 60, Validation Accuracy: 0.7561436672967864
Current Epoch: 60, Validation Accuracy: 0.8166351606805293
Current Epoch: 60, Validation Accuracy: 0.8162878787878788
Current Epoch: 60, Epoch Loss: 0.9507602453231812
Current Epoch: 61, Validation Accuracy: 0.8015122873345936
Current Epoch: 61, Validation Accuracy: 0.7561436672967864
Current Epoch: 61, Validat

Current Epoch: 87, Validation Accuracy: 0.8355387523629489
Current Epoch: 87, Validation Accuracy: 0.8185255198487713
Current Epoch: 87, Validation Accuracy: 0.8355387523629489
Current Epoch: 87, Validation Accuracy: 0.8390151515151515
Current Epoch: 87, Epoch Loss: 0.9076971411705017
Current Epoch: 88, Validation Accuracy: 0.8223062381852552
Current Epoch: 88, Validation Accuracy: 0.77882797731569
Current Epoch: 88, Validation Accuracy: 0.8771266540642723
Current Epoch: 88, Validation Accuracy: 0.8465909090909091
Current Epoch: 88, Epoch Loss: 0.9267500042915344
Current Epoch: 89, Validation Accuracy: 0.8128544423440454
Current Epoch: 89, Validation Accuracy: 0.8298676748582231
Current Epoch: 89, Validation Accuracy: 0.8676748582230623
Current Epoch: 89, Validation Accuracy: 0.8409090909090909
Current Epoch: 89, Epoch Loss: 0.9099558591842651
Current Epoch: 90, Validation Accuracy: 0.8279773156899811
Current Epoch: 90, Validation Accuracy: 0.7901701323251418
Current Epoch: 90, Validat

Current Epoch: 115, Validation Accuracy: 0.8901515151515151
Current Epoch: 115, Epoch Loss: 0.8802370429039001
Current Epoch: 116, Validation Accuracy: 0.8657844990548205
Current Epoch: 116, Validation Accuracy: 0.8657844990548205
Current Epoch: 116, Validation Accuracy: 0.8733459357277883
Current Epoch: 116, Validation Accuracy: 0.9053030303030303
Current Epoch: 116, Epoch Loss: 0.8814135193824768
Current Epoch: 117, Validation Accuracy: 0.8582230623818525
Current Epoch: 117, Validation Accuracy: 0.8374291115311909
Current Epoch: 117, Validation Accuracy: 0.8941398865784499
Current Epoch: 117, Validation Accuracy: 0.9071969696969697
Current Epoch: 117, Epoch Loss: 0.8802719116210938
Current Epoch: 118, Validation Accuracy: 0.8638941398865785
Current Epoch: 118, Validation Accuracy: 0.8620037807183365
Current Epoch: 118, Validation Accuracy: 0.8922495274102079
Current Epoch: 118, Validation Accuracy: 0.8920454545454546
Current Epoch: 118, Epoch Loss: 0.8624191880226135
Current Epoch: 1

...

Current Epoch: 881, Validation Accuracy: 0.9319470699432892
Current Epoch: 881, Validation Accuracy: 0.9716446124763705
Current Epoch: 881, Validation Accuracy: 0.9565217391304348
Current Epoch: 881, Validation Accuracy: 0.9621212121212122
Current Epoch: 881, Epoch Loss: 0.7922416925430298
Current Epoch: 882, Validation Accuracy: 0.9395085066162571
Current Epoch: 882, Validation Accuracy: 0.9754253308128544
Current Epoch: 882, Validation Accuracy: 0.9527410207939508
Current Epoch: 882, Validation Accuracy: 0.9564393939393939
Current Epoch: 882, Epoch Loss: 0.7919113636016846
Current Epoch: 883, Validation Accuracy: 0.941398865784499
Current Epoch: 883, Validation Accuracy: 0.9621928166351607
Current Epoch: 883, Validation Accuracy: 0.9621928166351607
Current Epoch: 883, Validation Accuracy: 0.9621212121212122
Current Epoch: 883, Epoch Loss: 0.7930818200111389
Current Epoch: 884, Validation Accuracy: 0.9376181474480151
Current Epoch: 884, Validation Accuracy: 0.9697542533081286
Current 

Current Epoch: 909, Validation Accuracy: 0.9754253308128544
Current Epoch: 909, Validation Accuracy: 0.9791666666666666
Current Epoch: 909, Epoch Loss: 0.7750774621963501
Current Epoch: 910, Validation Accuracy: 0.943289224952741
Current Epoch: 910, Validation Accuracy: 0.9792060491493384
Current Epoch: 910, Validation Accuracy: 0.9735349716446124
Current Epoch: 910, Validation Accuracy: 0.9753787878787878
Current Epoch: 910, Epoch Loss: 0.7758970856666565
Current Epoch: 911, Validation Accuracy: 0.945179584120983
Current Epoch: 911, Validation Accuracy: 0.9773156899810964
Current Epoch: 911, Validation Accuracy: 0.9773156899810964
Current Epoch: 911, Validation Accuracy: 0.9753787878787878
Current Epoch: 911, Epoch Loss: 0.7776812314987183
Current Epoch: 912, Validation Accuracy: 0.945179584120983
Current Epoch: 912, Validation Accuracy: 0.9792060491493384
Current Epoch: 912, Validation Accuracy: 0.9735349716446124
Current Epoch: 912, Validation Accuracy: 0.9715909090909091
Current Ep

Current Epoch: 938, Validation Accuracy: 0.9489603024574669
Current Epoch: 938, Validation Accuracy: 0.9792060491493384
Current Epoch: 938, Validation Accuracy: 0.9773156899810964
Current Epoch: 938, Validation Accuracy: 0.9810606060606061
Current Epoch: 938, Epoch Loss: 0.7751839756965637
Current Epoch: 939, Validation Accuracy: 0.9489603024574669
Current Epoch: 939, Validation Accuracy: 0.9792060491493384
Current Epoch: 939, Validation Accuracy: 0.9754253308128544
Current Epoch: 939, Validation Accuracy: 0.9772727272727273
Current Epoch: 939, Epoch Loss: 0.7731829285621643
Current Epoch: 940, Validation Accuracy: 0.9489603024574669
Current Epoch: 940, Validation Accuracy: 0.9754253308128544
Current Epoch: 940, Validation Accuracy: 0.9773156899810964
Current Epoch: 940, Validation Accuracy: 0.9753787878787878
Current Epoch: 940, Epoch Loss: 0.7719488143920898
Current Epoch: 941, Validation Accuracy: 0.9508506616257089
Current Epoch: 941, Validation Accuracy: 0.9735349716446124
Current

Current Epoch: 966, Validation Accuracy: 0.9754253308128544
Current Epoch: 966, Validation Accuracy: 0.9810606060606061
Current Epoch: 966, Epoch Loss: 0.7679785490036011
Current Epoch: 967, Validation Accuracy: 0.9508506616257089
Current Epoch: 967, Validation Accuracy: 0.9792060491493384
Current Epoch: 967, Validation Accuracy: 0.9697542533081286
Current Epoch: 967, Validation Accuracy: 0.9753787878787878
Current Epoch: 967, Epoch Loss: 0.7743483185768127
Current Epoch: 968, Validation Accuracy: 0.947069943289225
Current Epoch: 968, Validation Accuracy: 0.9773156899810964
Current Epoch: 968, Validation Accuracy: 0.9773156899810964
Current Epoch: 968, Validation Accuracy: 0.9791666666666666
Current Epoch: 968, Epoch Loss: 0.7799147367477417
Current Epoch: 969, Validation Accuracy: 0.9489603024574669
Current Epoch: 969, Validation Accuracy: 0.9792060491493384
Current Epoch: 969, Validation Accuracy: 0.9678638941398866
Current Epoch: 969, Validation Accuracy: 0.9772727272727273
Current 

Current Epoch: 994, Validation Accuracy: 0.9772727272727273
Current Epoch: 994, Epoch Loss: 0.7711275815963745
Current Epoch: 995, Validation Accuracy: 0.9508506616257089
Current Epoch: 995, Validation Accuracy: 0.9735349716446124
Current Epoch: 995, Validation Accuracy: 0.9735349716446124
Current Epoch: 995, Validation Accuracy: 0.9810606060606061
Current Epoch: 995, Epoch Loss: 0.7813349962234497
Current Epoch: 996, Validation Accuracy: 0.9489603024574669
Current Epoch: 996, Validation Accuracy: 0.9754253308128544
Current Epoch: 996, Validation Accuracy: 0.9735349716446124
Current Epoch: 996, Validation Accuracy: 0.9772727272727273
Current Epoch: 996, Epoch Loss: 0.7682700753211975
Current Epoch: 997, Validation Accuracy: 0.9508506616257089
Current Epoch: 997, Validation Accuracy: 0.9810964083175804
Current Epoch: 997, Validation Accuracy: 0.9754253308128544
Current Epoch: 997, Validation Accuracy: 0.9791666666666666
Current Epoch: 997, Epoch Loss: 0.779339075088501
Current Epoch: 99

In [12]:
X_test = torch.tensor(X_test.reshape(X_test.shape[0], -1, 22, 1000), dtype= torch.float).to(device)
y_test = torch.tensor(y_test, dtype = torch.long).to(device)
output_test = cnn(X_test)
prediction_test = torch.argmax(output_test, axis = 1)
accuracy_test = float(prediction_test.eq(y_test).sum()) / float(y_test.shape[0])
print('Current Epoch: {}, Test Accuracy: {}'.format(epoch, accuracy_test))

Current Epoch: 999, Test Accuracy: 0.6207674943566591


In [13]:
torch.save(cnn, 'model_different_filter.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(range(num_epoch), epoch_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('CNN with Fewer Parameters')
fig.savefig('CNN_Filter_50_Size_3_Stride.png', dpi=fig.dpi)

In [15]:
epoch_loss = [i.cpu() for i in epoch_loss]
epoch_loss = [i.detach().numpy() for i in epoch_loss]
np.save('epoch_loss_CNN_EEG_Filter_50_Size_3_Stride.npy', epoch_loss)

In [16]:
np.save('train_acc_CNN_EEG_Filter_50_Size_3_Stride.npy', train_accuracy)

In [17]:
np.save('val_acc_CNN_EEG_Filter_50_Size_3_Stride.npy', validation_accuracy)

In [18]:
train_accuracy

[0.3642159538685616,
 0.4288389825592714,
 0.47817088084062576,
 0.5178892816873542,
 0.5434208508443843,
 0.5525600699567975,
 0.570212659446909,
 0.584397901931758,
 0.5965323748838888,
 0.6140276330939196,
 0.621436804077264,
 0.6242710516006869,
 0.645548617352051,
 0.655005578109021,
 0.6633585381222432,
 0.6731310553671024,
 0.6758095608152939,
 0.6802215907781621,
 0.6748625933757174,
 0.6832143614853026,
 0.6873136160687681,
 0.6888915971588196,
 0.6904666978150817,
 0.6972455504250726,
 0.7139480933912121,
 0.6915683147515557,
 0.7128454832017075,
 0.7147359416952526,
 0.7213568670733441,
 0.7156829091348289,
 0.7128457811776168,
 0.7164685722822014,
 0.7286054290416062,
 0.7200974222302742,
 0.7193078853960815,
 0.7211997344438696,
 0.7117441642411427,
 0.7218280663111615,
 0.739481649054304,
 0.7382189264762322,
 0.7503575710910925,
 0.738694297376779,
 0.7561874697554452,
 0.7554013099815573,
 0.7594968895288087,
 0.7642265618109307,
 0.7673773590752735,
 0.7687957244032735

In [19]:
validation_accuracy

[0.35225035086211837,
 0.4378338560462851,
 0.4799140387810047,
 0.519167633327605,
 0.5404431245345707,
 0.5621822549693533,
 0.5664498840006873,
 0.5834666967405624,
 0.5967027911439536,
 0.6118355101105574,
 0.6165569327490406,
 0.6373795254052815,
 0.6510819442057627,
 0.6543855974680644,
 0.662430364610185,
 0.6652623231368506,
 0.6841793406656356,
 0.6685749269633958,
 0.6747212794294553,
 0.669994486452426,
 0.6855962149853927,
 0.6960012459185427,
 0.6888962880219969,
 0.701194363292662,
 0.7002500787649653,
 0.7068761814744802,
 0.7120701939050238,
 0.7149119980523573,
 0.7111196439823566,
 0.7097036647190239,
 0.707803459930114,
 0.7215237798590823,
 0.7243700592885375,
 0.7196289454087186,
 0.7163235020335681,
 0.7300241307212006,
 0.7087602752477516,
 0.7205768101621126,
 0.7399646273701094,
 0.7456392851005327,
 0.7309764707567166,
 0.7295649667755055,
 0.7465790943461076,
 0.7517820573408948,
 0.7574540299020449,
 0.76124727902847,
 0.7673954216073781,
 0.7702291702468924